In [2]:
# 1) Prepara datos (año, pivot por país y estándar de tecnologías)
import pandas as pd

df = pd.read_csv("Isea/Energy_clean.csv")
year_cols = [c for c in df.columns if c.startswith("F")]
for c in year_cols: df[c] = pd.to_numeric(df[c], errors="coerce")

IND, UNIT = "Electricity Installed Capacity", "Megawatt (MW)"
f = df[(df["Indicator"]==IND) & (df["Unit"]==UNIT)].copy()
YEAR = [c for c in year_cols if f[c].notna().any()][-1]

tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
f["Technology_std"] = f["Technology"].map(tech_map).fillna(f["Technology"])

# Pivot por país (1 punto = país) para el scatter
idx = pd.MultiIndex.from_product([f["Country"].dropna().unique(),
                                  sorted(f["Technology_std"].dropna().unique())],
                                 names=["Country","Technology_std"])
grid = (f.groupby(["Country","Technology_std"])[YEAR].sum(min_count=1)
          .reindex(idx, fill_value=0.0).reset_index())
piv = (grid.pivot(index="Country", columns="Technology_std", values=YEAR)
          .fillna(0.0).reset_index())

for c in ["Solar","Wind","Hydro","Bio","Fossil"]:
    if c not in piv.columns: piv[c] = 0.0
piv["Total"] = piv.drop(columns=["Country"]).sum(axis=1, numeric_only=True)
piv["DominantTech"] = piv[["Solar","Wind","Hydro","Bio","Fossil"]].idxmax(axis=1)

# Para que no se amontonen, quitamos (0,0) solo del SCATTER:
piv_viz = piv[(piv["Solar"]>0) | (piv["Wind"]>0)].copy()
print(f"Año: {YEAR} | países con Solar/Wind>0: {len(piv_viz)}  | filas totales CSV: {len(f)}")

Año: F2023 | países con Solar/Wind>0: 82  | filas totales CSV: 570


In [3]:
from Isea import ScatterBrush
from IPython.display import display
from ipywidgets import Output
import pandas as pd

# sample data (note the actual column names)
df = pd.DataFrame({
    "id": [1,2,3,4],
    "x":  [1,2,3,4],
    "y":  [10,8,6,9],
    "group": ["A","B","A","B"]   # <-- categorical for color
})

# instantiate with the CORRECT column names
w = ScatterBrush(
    df,
    x="x", y="y",           # <-- must match df
    color="group",          # optional, only if it exists
    key="id",               # optional; defaults to a stable id anyway
    title="My Scatter", x_label="X", y_label="Y",
    width=720, height=420
)
display(w)

# show selection live below the chart
out = Output()
def on_sel(change):
    sel = change["new"]
    with out:
        out.clear_output()
        if sel.get("type"):
            rows = sel.get("rows", [])
            print(f"{sel['type']} selection — {len(rows)} points")
            display(pd.DataFrame(rows))
        else:
            print("Selection cleared")
w.observe(on_sel, names="selection")
display(out)


ScatterBrush(data=[{'id': 1, 'x': 1, 'y': 10, 'group': 'A'}, {'id': 2, 'x': 2, 'y': 8, 'group': 'B'}, {'id': 3…

Output()

In [4]:
display (out)

Output()

In [5]:
import pandas as pd
from Isea import ParallelEnergy

df = pd.read_csv("Isea/Energy_clean.csv")

years = [c for c in df.columns if c.startswith("F")]
tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
df["Technology_std"] = df["Technology"].map(tech_map).fillna(df["Technology"])


In [6]:
w = ParallelEnergy(
    df, years,
    tech_col="Technology_std",
    label_col="Country",
    dims=("Fossil", "Solar", "Hydro", "Wind", "Bio"),
    year_start="F2023",
    width=1100, height=560,
    margin=dict(left=50, right=180, top=80, bottom=36),  # ajusta si quieres
    panel_position="bottom",       # "right" o "bottom"
    panel_width=320, panel_height=220,
    log_axes=False, normalize=False, reorder=True
)
w


ParallelEnergy(data={'years': ['F2000', 'F2001', 'F2002', 'F2003', 'F2004', 'F2005', 'F2006', 'F2007', 'F2008'…

In [6]:
w.show_selection()

,Country,Year,Fossil,Solar,Hydro,Wind,Bio,DominantTech
0,Australia,F2023,50820.000,32612.000,8439.960,12911.000,871.360,Fossil
1,Austria,F2023,4092.153,6831.704,14708.126,3977.236,1195.101,Hydro
2,Denmark,F2023,5255.738,3529.000,6.644,7481.818,2006.594,Wind
3,France,F2023,18499.104,20551.293,24139.006,20810.575,2187.252,Hydro
4,Germany,F2023,88934.000,81739.000,5741.000,69459.000,9950.000,Fossil
5,India,F2023,0.000,73109.342,47335.995,44736.240,10752.414,Solar
6,Italy,F2023,57284.374,29795.318,18847.790,12307.508,3434.457,Fossil
7,Japan,F2023,0.000,89077.000,28129.000,5343.000,5805.000,Solar
8,"Korea, Rep. of",F2023,90056.972,23493.085,1804.996,2204.860,2745.500,Fossil
9,Mexico,F2023,64909.500,10909.793,12838.009,7317.720,886.120,Fossil


,Country,Year,Fossil,Solar,Hydro,Wind,Bio,DominantTech
0,Australia,F2023,50820.000,32612.000,8439.960,12911.000,871.360,Fossil
1,Austria,F2023,4092.153,6831.704,14708.126,3977.236,1195.101,Hydro
2,Denmark,F2023,5255.738,3529.000,6.644,7481.818,2006.594,Wind
3,France,F2023,18499.104,20551.293,24139.006,20810.575,2187.252,Hydro
4,Germany,F2023,88934.000,81739.000,5741.000,69459.000,9950.000,Fossil
5,India,F2023,0.000,73109.342,47335.995,44736.240,10752.414,Solar
6,Italy,F2023,57284.374,29795.318,18847.790,12307.508,3434.457,Fossil
7,Japan,F2023,0.000,89077.000,28129.000,5343.000,5805.000,Solar
8,"Korea, Rep. of",F2023,90056.972,23493.085,1804.996,2204.860,2745.500,Fossil
9,Mexico,F2023,64909.500,10909.793,12838.009,7317.720,886.120,Fossil


In [7]:
from IPython.display import display, clear_output
import ipywidgets as ipw

out = ipw.Output()
display(out)

def on_sel(change):
    with out:
        clear_output(wait=True)
        df_sel = w.selection_df()
        print(f"point selection — {len(df_sel)} points")
        display(df_sel)

# Escucha cambios del widget
w.observe(on_sel, "selection")


Output()

In [8]:
w2 = w.new_from_selection(
    width=1000, height=520,
    margin=dict(l=48, r=160, t=72, b=32),
    panel_position="bottom",
    panel_height=200
)
w2


ParallelEnergy(data={'years': ['F2000', 'F2001', 'F2002', 'F2003', 'F2004', 'F2005', 'F2006', 'F2007', 'F2008'…

In [11]:
import inspect
from Isea.radial_stacked_bar import RadialStackedBar

# Ver la firma de la función
print(inspect.signature(RadialStackedBar.__init__))

# Ver si year_cols está en los parámetros
params = inspect.signature(RadialStackedBar.__init__).parameters
print("year_cols en parámetros:", "year_cols" in params)

(self, df: pandas.core.frame.DataFrame, group_col: str, category_col: str, year_cols: Optional[List[str]] = None, year_start: Optional[str] = None, agg: str = 'sum', width: int = 900, height: int = 900, inner_radius: int = 200, pad_angle: float = 0.015, color_scheme: str = 'schemeSpectral', sort_on_click: bool = True, title: Optional[str] = None, custom_colors: Optional[List[str]] = None, **kwargs)
year_cols en parámetros: True


In [17]:
# ========== PRIMERO: Preparar datos (IGUAL A LA CELDA 1) ==========
import pandas as pd
from Isea import RadialStackedBar

df = pd.read_csv("Isea/Energy_clean.csv")
year_cols = [c for c in df.columns if c.startswith("F")]

# Convertir a numéricas
for c in year_cols: 
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Filtrar por indicador y unidad
IND, UNIT = "Electricity Installed Capacity", "Megawatt (MW)"
df = df[(df["Indicator"]==IND) & (df["Unit"]==UNIT)].copy()

# IMPORTANTE: Estandarizar tecnologías
tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
df["Technology_std"] = df["Technology"].map(tech_map).fillna(df["Technology"])

print(f"✅ Datos preparados: {len(df)} filas")
print(f"✅ Columnas: {df.columns.tolist()}")
print(f"✅ Tecnologías únicas: {df['Technology_std'].unique()}")

# ========== AHORA: Crear RadialStackedBar ==========
radial = RadialStackedBar(
    df=df,
    group_col="Country",
    category_col="Technology_std",
    year_cols=year_cols,
    year_start="F2023",
    width=900,
    height=900,
    inner_radius=200,
    title="⚡ Capacidad Instalada",
    custom_colors=["#d73027","#f46d43","#fdae61","#fee08b","#e6f598","#abdda4","#66c2a5","#3288bd"]
)

display(radial)
print("✅ Widget creado exitosamente!")

✅ Datos preparados: 570 filas
✅ Columnas: ['ObjectId', 'Country', 'ISO2', 'ISO3', 'Indicator', 'Technology', 'Energy_Type', 'Unit', 'Source', 'CTS_Name', 'CTS_Code', 'CTS_Full_Descriptor', 'F2000', 'F2001', 'F2002', 'F2003', 'F2004', 'F2005', 'F2006', 'F2007', 'F2008', 'F2009', 'F2010', 'F2011', 'F2012', 'F2013', 'F2014', 'F2015', 'F2016', 'F2017', 'F2018', 'F2019', 'F2020', 'F2021', 'F2022', 'F2023', 'Technology_std']
✅ Tecnologías únicas: ['Fossil' 'Hydro' 'Bio' 'Solar' 'Wind']


RadialStackedBar(data={'years': ['F2000', 'F2001', 'F2002', 'F2003', 'F2004', 'F2005', 'F2006', 'F2007', 'F200…

✅ Widget creado exitosamente!


In [14]:
print("Columns:", df.columns.tolist())
print("Has Technology_std:", "Technology_std" in df.columns)
print("Year cols:", year_cols)
print("Sample data:\n", df[["Country", "Technology_std", year_cols[0]]].head())

Columns: ['ObjectId', 'Country', 'ISO2', 'ISO3', 'Indicator', 'Technology', 'Energy_Type', 'Unit', 'Source', 'CTS_Name', 'CTS_Code', 'CTS_Full_Descriptor', 'F2000', 'F2001', 'F2002', 'F2003', 'F2004', 'F2005', 'F2006', 'F2007', 'F2008', 'F2009', 'F2010', 'F2011', 'F2012', 'F2013', 'F2014', 'F2015', 'F2016', 'F2017', 'F2018', 'F2019', 'F2020', 'F2021', 'F2022', 'F2023', 'Technology_std']
Has Technology_std: True
Year cols: ['F2000', 'F2001', 'F2002', 'F2003', 'F2004', 'F2005', 'F2006', 'F2007', 'F2008', 'F2009', 'F2010', 'F2011', 'F2012', 'F2013', 'F2014', 'F2015', 'F2016', 'F2017', 'F2018', 'F2019', 'F2020', 'F2021', 'F2022', 'F2023']
Sample data:
                         Country Technology_std     F2000
0  Afghanistan, Islamic Rep. of         Fossil    29.725
1  Afghanistan, Islamic Rep. of          Hydro   191.503
2                       Albania          Hydro  1453.000
3                       Algeria         Fossil  5846.210
4                       Algeria          Hydro   276.600


In [15]:
# Fuerza recarga
import sys
for mod in list(sys.modules.keys()):
    if 'Isea' in mod:
        del sys.modules[mod]

# Reimporta
from Isea import RadialStackedBar
from IPython.display import display

# Preparar datos (mismo que antes)
import pandas as pd

df = pd.read_csv("Isea/Energy_clean.csv")
year_cols = [c for c in df.columns if c.startswith("F")]

for c in year_cols: 
    df[c] = pd.to_numeric(df[c], errors="coerce")

IND, UNIT = "Electricity Installed Capacity", "Megawatt (MW)"
df = df[(df["Indicator"]==IND) & (df["Unit"]==UNIT)].copy()

tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
df["Technology_std"] = df["Technology"].map(tech_map).fillna(df["Technology"])

# Crear widget
radial = RadialStackedBar(
    df=df,
    group_col="Country",
    category_col="Technology_std",
    year_cols=year_cols,
    year_start="F2023",
    width=900,
    height=900,
    inner_radius=200,
    title="⚡ Capacidad Instalada",
    custom_colors=["#d73027","#f46d43","#fdae61","#fee08b","#e6f598","#abdda4","#66c2a5","#3288bd"]
)

display(radial)
print("✅ Widget visualizado!")

RadialStackedBar(data={'years': ['F2000', 'F2001', 'F2002', 'F2003', 'F2004', 'F2005', 'F2006', 'F2007', 'F200…

✅ Widget visualizado!


In [16]:
from pathlib import Path
from Isea import RadialStackedBar

# Verificar que _esm esté configurado
print("_esm path:", RadialStackedBar._esm)
print("Existe:", Path(RadialStackedBar._esm).exists())

# Si no existe, dinos y crearemos el archivo
print("\nContenido esperado del archivo:")
import inspect
print(inspect.getfile(RadialStackedBar))

_esm path: // RadialStackedBar ESM module para anywidget

export function render({ model, el }) {
  // ========== Crear estructura HTML ==========
  const container = document.createElement("div");
  container.style.cssText = `
    display: flex;
    flex-direction: column;
    gap: 12px;
    padding: 16px;
  `;

  // Título
  const title = document.createElement("div");
  title.style.cssText = `
    font-size: 16px;
    font-weight: 600;
    color: #1f2937;
  `;
  container.appendChild(title);

  // Barra de controles
  const controls = document.createElement("div");
  controls.style.cssText = `
    display: flex;
    gap: 12px;
    align-items: center;
    padding: 8px;
    background: #f9fafb;
    border-radius: 8px;
  `;

  const yearLabel = document.createElement("span");
  yearLabel.textContent = "Año:";
  yearLabel.style.cssText = `font-weight: 500; color: #374151;`;
  controls.appendChild(yearLabel);

  const slider = document.createElement("input");
  slider.type = "range";
  

TypeError: argument should be a str or an os.PathLike object where __fspath__ returns a str, not 'FileContents'

In [21]:
# ========== PREPARACIÓN AGRESIVA DE DATOS ==========
import pandas as pd
from Isea import RadialStackedBar

df = pd.read_csv("Isea/Energy_clean.csv")
year_cols = [c for c in df.columns if c.startswith("F")]

# Convertir a numéricas
for c in year_cols: 
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Filtrar por indicador y unidad
IND, UNIT = "Electricity Installed Capacity", "Megawatt (MW)"
df = df[(df["Indicator"]==IND) & (df["Unit"]==UNIT)].copy()

# Estandarizar tecnologías
tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
df["Technology_std"] = df["Technology"].map(tech_map).fillna(df["Technology"])

# ========== Filtrar solo tecnologías estándar ==========
df_clean = df[
    df["Technology_std"].isin(["Fossil", "Solar", "Wind", "Hydro", "Bio"])
].copy()

# ========== NUEVO: Agrupar por país y tecnología ==========
df_grouped = df_clean.groupby(["Country", "Technology_std"])[year_cols].sum().reset_index()

# ========== NUEVO: Filtrar países con TODAS las tecnologías ==========
# Contar cuántas tecnologías tiene cada país
tech_count = df_grouped.groupby("Country")["Technology_std"].nunique()

# Mantener solo países con LAS 5 TECNOLOGÍAS
required_techs = 5  # Fossil, Solar, Wind, Hydro, Bio
complete_countries = tech_count[tech_count == required_techs].index.tolist()

print(f"📊 Países totales: {df_grouped['Country'].nunique()}")
print(f"✅ Países con 5 tecnologías: {len(complete_countries)}")

# Filtrar
df_grouped = df_grouped[df_grouped["Country"].isin(complete_countries)].copy()

# ========== NUEVO: Eliminar filas donde TODOS los años son 0 ==========
df_grouped["Total"] = df_grouped[year_cols].sum(axis=1)
df_grouped = df_grouped[df_grouped["Total"] > 0].drop(columns=["Total"])

# ========== NUEVO: Filtrar por año más reciente con datos ==========
# Solo mantener países con datos en F2023 (año más reciente)
last_year = "F2023"
df_grouped = df_grouped[df_grouped[last_year] > 0].copy()

print(f"✅ Datos finales después de filtrar: {len(df_grouped)} filas")
print(f"✅ Países únicos: {df_grouped['Country'].nunique()}")
print(f"✅ Tecnologías: {df_grouped['Technology_std'].unique()}")

# ========== Crear RadialStackedBar ==========
radial = RadialStackedBar(
    df=df_grouped,
    group_col="Country",
    category_col="Technology_std",
    year_cols=year_cols,
    year_start="F2023",
    width=900,
    height=900,
    inner_radius=200,
    title="⚡ Capacidad Instalada (MW) - Países Completos",
    custom_colors=["#d73027","#f46d43","#fdae61","#fee08b","#e6f598","#abdda4","#66c2a5","#3288bd"]
)

display(radial)
print("✅ Widget creado con datos limpios!")

📊 Países totales: 216
✅ Países con 5 tecnologías: 26
✅ Datos finales después de filtrar: 130 filas
✅ Países únicos: 26
✅ Tecnologías: ['Bio' 'Fossil' 'Hydro' 'Solar' 'Wind']


RadialStackedBar(data={'years': ['F2000', 'F2001', 'F2002', 'F2003', 'F2004', 'F2005', 'F2006', 'F2007', 'F200…

✅ Widget creado con datos limpios!
